In [489]:
#libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import normalize

In [490]:
#read the data 
recipes = pd.read_csv('../data/full_clean_data.csv')
#rename the index
recipes.index.names = ['recipes_id']
recipes.tail(2)

name  \
recipes_id                              
19222                 Jalapeno Hummus   
19223       Easy Baked Zucchini Chips   

                                                          url  \
recipes_id                                                      
19222       https://www.allrecipes.com/recipe/46462/jalape...   
19223       https://www.allrecipes.com/recipe/242066/easy-...   

                         category  rating  rating_count  \
recipes_id                                                
19222       appetizers-and-snacks    4.56           281   
19223       appetizers-and-snacks    3.68            28   

                                                  ingredients  total  \
recipes_id                                                             
19222       ['1 cup garbanzo beans ', ' ⅓ cup canned jalap...     10   
19223       ['1  zucchini, thinly sliced ', ' 1 tablespoon...     30   

            servings       yield  calories  carbohydrates_g  fat_g  protein_g  \
recipes_id                                                                      
19222              8  8 servings      74.6              9.1    3.5        2.6   
19223              2  2 servings      69.1              2.0    6.9        0.7   

                                           ingredients_parsed  
recipes_id                                                     
19222       ['garbanzo bean', 'jalapeno reserved', 'tahini...  
19223                                            ['flavored']

In [491]:
##create a new df for user input & concate it to the original dataframe
#user input: a = total, b = calories, c = carbohydrate, d = fat, e = protein.
def user_recipes(a, b, c, d, e):
    user_df = pd.DataFrame({'recipes_id': 19224 ,'name': 'user', 'url': 'no', 'category': 'no', 'rating': 'no', 'rating_count': 'no', 'ingredients': 'no', 'total': a, 'servings': 'no', 'yield': 'no', 'calories': b, 'carbohydrates_g': c, 'fat_g': d,'protein_g': e,'ingredients_parsed': 'no'}, index=[19224])
    full_df = pd.concat([recipes, user_df], ignore_index = False, axis = 0) 
    #rename the dataframe index
    full_df.index.names = ['recipes_id']
    ##convert index to column
    full_df['recipes_id'] = full_df.index
    ##normalize the total, calories, carb, fat, protein features
    new_norm = full_df[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
    full_norm = pd.DataFrame(normalize(new_norm, axis=0))
    full_norm.columns = new_norm.columns
    full_norm.index = new_norm.index
    print(full_norm.tail(2))
    #use the model function here
    allRecipes = pd.DataFrame(full_norm.index)
    allRecipes = allRecipes[allRecipes.recipes_id != recipes_id]
    allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: euclidean(full_norm.loc[recipes_id], full_norm.loc[19224]))
    # sort by distance then recipe id, the smaller value of recipe id will be picked.
    Top3NRecommendation = allRecipes.sort_values(["distance"]).head(3).sort_values(by=['distance', 'recipes_id'])
    #create a list of the recommended recipe output
    recipes_id = [recipes_id]
    recipe_list = []
    for recipeid in Top3NRecommendation.recipes_id:
        recipes_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe
        recipe_list.append("{}  {}".format(recipeid, full_norm.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, full_norm.at[recipeid, 'name']))
    return recipe_list
    

In [492]:
user_recipes(45, 300, 40, 9, 8)

               total  calories  carbohydrates_g     fat_g  protein_g
recipes_id                                                          
19223       0.001256   0.00164         0.000412  0.003421   0.000298
19224       0.001884   0.00712         0.008230  0.004462   0.003407


UnboundLocalError: local variable 'recipes_id' referenced before assignment